### Random Recommendation Baseline,
    This notebook implements a Random Recommendation baseline model for the Yelp dataset. It recommends a random set of businesses to each user and evaluates the performance using retrieval metrics.
    
#### Pre-requisites,
- The Yelp dataset is loaded from the processed data folder (`../../data/processed_data/yelp_data/`).

In [15]:
import sys
sys.path.append('../')
from utilities import *

In [16]:
# Define the database folder path and file names
db_folder = '../../data/processed_data/yelp_data/'
db_files = ['yelp_business_data.db', 'yelp_review_data.db']
db_paths = [db_folder + db_file for db_file in db_files]

In [17]:
# Connect to the databases and load data
def load_data_from_db():
    data = {}
    conns = [sqlite3.connect(db_path) for db_path in db_paths]
    try:
        data['business'] = pd.read_sql_query("SELECT * FROM business_details", conns[0])
        data['review'] = pd.read_sql_query("SELECT * FROM review_data", conns[1])
    finally:
        for conn in conns:
            conn.close()
    return data

In [18]:
yelp_data = load_data_from_db()

for table, df in yelp_data.items():
    print(f"Loaded {len(df)} rows from {table} table.")

Loaded 78059 rows from business table.
Loaded 980418 rows from review table.


In [19]:
df_business = yelp_data["business"]
df_review = yelp_data["review"]

In [20]:
df_concat = df_business.merge(df_review, on='business_id', how='outer', suffixes=('_business', '_review'))
user_business = df_concat[["user_id", "business_id", "stars_review"]]

all_business_ids = user_business['business_id'].unique()

In [21]:
train_data, test_data = train_test_split(user_business, test_size=0.2, random_state=42)

In [22]:
def predict_random_interests(user_id, k=10, all_business_ids=all_business_ids):
    recommended_businesses = np.random.choice(all_business_ids, size=k, replace=False)
    scores = np.random.uniform(0, 1, size=k)
    return list(zip(recommended_businesses, scores))

In [23]:
positive_reviews = test_data[test_data['stars_review'] >= 4]
negative_reviews = test_data[test_data['stars_review'] <= 2]

In [24]:
print(f"Number of positive reviews: {len(positive_reviews)}")
print(f"Number of negative reviews: {len(negative_reviews)}")
print(f"Total number of reviews: {len(test_data)}")
print(f"Ratio of positive to negative reviews: {len(positive_reviews) / len(negative_reviews):.2f}")

Number of positive reviews: 136473
Number of negative reviews: 32929
Total number of reviews: 197147
Ratio of positive to negative reviews: 4.14


In [25]:
test_data_grouped = test_data.groupby('user_id')['business_id'].apply(list).reset_index()

In [26]:
recommendations = {}
k_recommendations = 300
max_users = min(1000, len(test_data_grouped))

In [27]:
for i in range(max_users):
    user_id = test_data_grouped['user_id'].iloc[i]
    recommendation = predict_random_interests(user_id, k=k_recommendations)
    business_ids, scores = zip(*recommendation)
    recommendations[user_id] = (list(business_ids), list(scores))

In [29]:
def check_recommendations(recommendations, test_data_grouped, pos=4):
    total = 0
    total_positive = 0
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    ranks = []
    for i, row in test_data_grouped.iterrows():
        user_id = row['user_id']
        business_ids = row['business_id']
        rank = 0
        if user_id in recommendations:
            recommended_businesses = recommendations[user_id][0]
            for business_id in business_ids:
                star_rating = test_data[(test_data['user_id'] == user_id) & 
                                       (test_data['business_id'] == business_id)]['stars_review'].values[0]
                if star_rating >= pos:
                    total_positive += 1
                if business_id in recommended_businesses:
                    if star_rating >= pos:
                        true_positive += 1
                    else:
                        false_positive += 1
                    rank = recommended_businesses.index(business_id) + 1
                else:
                    if star_rating < pos:
                        true_negative += 1
                    else:
                        false_negative += 1
            total += len(business_ids)
        ranks.append(rank)
    return true_positive, true_negative, false_positive, false_negative, total, total_positive, ranks

In [30]:
true_positive, true_negative, false_positive, false_negative, total, total_positive, ranks = check_recommendations(recommendations, test_data_grouped)

In [31]:
evaluation_metric, confusion_matrix, background_stats = compute_evaluation_metric(true_positive, true_negative, false_positive, false_negative, total, total_positive, ranks)

In [32]:
print("Testing Data Statistics")
display(background_stats)

print("Evaluation Metrics")
display(evaluation_metric)

print("Confusion Matrix")
display(confusion_matrix)

Testing Data Statistics


,Total Positive,Total Negative,Total,Ratio
0,1821,813,2634,0.691344


Evaluation Metrics


,Accuracy,Precision,Recall,F1 Score,F-beta Score,Mean Reciprocal Rank
0,0.3102,0.7,0.0038,0.0076,0.0048,0.0164


Confusion Matrix


,True Positive,True Negative,False Positive,False Negative
0,7,810,3,1814
